# simbio @ COMBINE 2023

Run the following cell to install `simbio` and `numba`,
which we will use in this tutorial.

In [ ]:
!pip install simbio[io] numba

It should also install `poincare`, `symbolite`, and other dependencies.

In [ ]:
import matplotlib.pyplot as plt

plt.rc("figure", figsize=(6, 2))  # smaller default figure height§

## poincaré

Poincaré lets us define dynamical systems.
Let's start with a simple exponential decay.

### Example 1: first order system

Exponential decay: $\frac{dx}{dt} = -x$

#### Defining a model

- Models are defined as subclasses of `System`
- Variables are annotated with `Variable` and assigned an initial value with `initial`.

In [ ]:
from poincare import Variable, System, initial


class Decay(System):
    x: Variable = initial(default=1)
    eq = x.derive() << -x


Decay

#### Simulating a model

To simulate a model,
we create an instance of a `Simulator`:

In [ ]:
from poincare import Simulator

sim = Simulator(Decay)

and call its `.solve` method:

In [ ]:
import numpy as np

df = sim.solve(save_at=np.linspace(0, 10, 100))

df  # this is just a pandas.DataFrame

As the output is a `pandas.DataFrame`,
we can easily operate on it with the many packages that use `DataFrame` as inputs,
or plot with its `plot` method:

In [ ]:
df.plot()

#### Changing initial values

To change initial values,
we ca pass a dictionary to its `solve` method:

In [ ]:
sim.solve(
    values={Decay.x: 2},
    save_at=np.linspace(0, 10, 100),
).plot()

Note that we use the model's attribute as dictionary key,
so the IDE's can help us with autocomplete, renaming, non-existing attributes, etc:

In [ ]:
class RenameMe(System):
    rename_me_too: Variable = initial(default=1)
    eq = rename_me_too.derive() << -rename_me_too


Simulator(RenameMe).solve(
    values={RenameMe.rename_me_too: 2},
    save_at=np.linspace(0, 1, 3),
)

### Example 2: second order system

Harmonic oscillator: $\frac{d^2x}{dt^2} = -x$

While it may not be needed for biological simulations,
it is needed for other fields, such as physics.

We can create a `Derivative` with the `Variable.derive` method:

In [ ]:
from poincare import Derivative


class Oscillator(System):
    x: Variable = initial(default=1)
    v: Derivative = x.derive(initial=1)
    eq = v.derive() << -x


Simulator(Oscillator).solve(save_at=np.linspace(0, 10, 1000)).plot()

#### Frequency as a parameter

$\frac{d^2x}{dt^2} = -w^2 x$

We can assign a `Parameter` with the `assign` function.

In [ ]:
from poincare import assign, Parameter


class Oscillator(System):
    x: Variable = initial(default=1)
    v: Derivative = x.derive(initial=1)
    w: Parameter = assign(default=1) 
    eq = v.derive() << -(w**2) * x


Simulator(Oscillator).solve(save_at=np.linspace(0, 10, 1000)).plot()

#### Interactive widgets

Using `ipywidgets` behind the scenes,
`Simulator.interact` creates widgets to interactively change parameters and initial values:

In [ ]:
Simulator(Oscillator).interact(save_at=np.linspace(0, 10, 1000))

We can choose to create widgets for only a subset of variables:

In [ ]:
Simulator(Oscillator).interact(
    values=[Oscillator.w],
    save_at=np.linspace(0, 10, 1000),
)

Or customize the range of values with `(start, stop, [step])`
or an instance of `ipywidgets.Widget`:

In [ ]:
Simulator(Oscillator).interact(
    values={Oscillator.w: (0, 10, 0.1)},  # or ipywidgets.FloatSlider(...)
    save_at=np.linspace(0, 10, 1000),
)

#### Transform

Sometimes we are just interested in a subset of variables from the simulation,
or a particular transformation.

*Note:
currently this is just implemented as a post-solve transformation,
but it could also help to reduce the memory footprint of simulations
(SciPy's ODE solvers do not yet support this).*

##### To observe a subset of variables

We pass a list of variables to the `Simulator`'s `transform` parameter:

In [ ]:
Simulator(
    Oscillator,
    transform=[Oscillator.x],
).interact(save_at=np.linspace(0, 10, 1000))

##### To observe a calculation

We can pass a symbolic expression:

In [ ]:
Simulator(
    Oscillator,
    transform=[Oscillator.x, Oscillator.x**2],
).interact(save_at=np.linspace(0, 10, 1000))

##### To rename output

We can pass a dictionary,
instead of a list,
where the keys are the new column names:

In [ ]:
Simulator(
    Oscillator,
    transform={
        "x": Oscillator.x,
        "Potential": Oscillator.x**2,
        "Kinetic": Oscillator.v**2 / Oscillator.w**2,
        "Energy": Oscillator.x**2 + Oscillator.v**2 / Oscillator.w**2
    },  # they are "densities", Energy per unit of mass
).interact(save_at=np.linspace(0, 10, 1000))

### Example 3: forced damped oscillator

Harmonic oscillator: $\frac{d^2x}{dt^2} = -x - b\frac{dx}{dt} + \cos(w t)$

We have:
- the restitutive term ($-x$)
- damping term $- b\frac{dx}{dt}$
- and the forcing term $cos(w t)$

This system will be in resonance for $w = 1$.

#### Independent variable

We will create an independent variable (i.e., time)
that we can use in the forcing equation.

Also,
we will use symbolic functions from `symbolite`.

In [ ]:
from poincare import Independent
from symbolite import scalar as f


class Oscillator(System):
    t = Independent()

    x: Variable = initial(default=1)
    v: Derivative = x.derive(initial=0)

    forcing_freq: Parameter = assign(default=0)
    damping: Parameter = assign(default=0.01)

    eq_restitutive = v.derive() << -x
    eq_damping = v.derive() << -damping * v
    eq_forced = v.derive() << f.sin(forcing_freq * t)


Simulator(
    Oscillator,
    transform=[Oscillator.x],
).interact(
    values={
        Oscillator.forcing_freq: (0., 1.5, 0.01),
        Oscillator.damping: (0, 1, 0.01),
    },
    save_at=np.linspace(0, 200, 1000),
)

### Example 4: composing models

We can create instances of other models to compose them into bigger ones.

(that's why we've been adding type annotations:
it generates the signature automatically)

Here we create two instances of the same `Decay` model
with different `decay_rate`s.

In [ ]:
class Decay(System):
    x: Variable = initial(default=1)
    decay_rate: Parameter = assign(default=1)
    eq = x.derive() << -decay_rate * x


class BigModel(System):
    decay_1 = Decay(decay_rate=1)
    decay_2 = Decay(decay_rate=2)


Simulator(BigModel).solve(save_at=np.linspace(0, 5, 1000)).plot()

There is an inner variables created in each model.

But we can also "link" those variables to an existing one,
instead of creating new ones:

In [ ]:
class BigModel(System):
    x: Variable = initial(default=1)
    decay_1 = Decay(x=x, decay_rate=1)
    decay_2 = Decay(x=x, decay_rate=2)


Simulator(BigModel).solve(save_at=np.linspace(0, 5, 1000)).plot()

### Example 5: Units

Last, but not least,
`poincaré` supports units out of the box using the `pint` package.

In [ ]:
import pint

u = pint.get_application_registry()


class Model(System):
    x: Variable = initial(default=1 * u.m)
    tau: Parameter = assign(default=1 * u.s)

    eq = x.derive() << x / tau  # try removing "/ tag"

## simbio

simbio adds on top of poincaré:

- `Species`, which is a `Variable` with a stoichiometry,
- `Reaction`, which generates the equations taking into account the stoichiometry
- `MassAction`, a particular type of reaction
- several predefined reactions based on `MassAction` (`Creation`, etc)
- (not yet implmenented) a `Volume` variable

### Compartment and Reactions

In simbio,
instead of creating a `System`,
we create a `Compartment`:

In [ ]:
from simbio import Species, Reaction, Compartment, initial, Simulator


class Model(Compartment):
    A: Species = initial(default=1)
    B: Species = initial(default=0)
    r = Reaction(reactants=[A], products=[B], rate_law=1)


Simulator(Model).solve(save_at=np.linspace(0, 1, 100)).plot()

### MassAction reaction

If we use `MassAction`,
it adds all reactants (with their stoichiometric exponents) to the `rate_law`:

In [ ]:
from simbio import MassAction


class Model(Compartment):
    A: Species = initial(default=1)
    B: Species = initial(default=0)
    r = MassAction(reactants=[A], products=[B], rate=1)
    # r = Reaction(reactants=[A], products=[B], rate=1 * A)


Simulator(Model).solve(save_at=np.linspace(0, 10, 100)).plot()

### Reactions with stoichiometries

To define a reaction with different stoichiometries,
just multiply the corresponding species by the stoichiometric factor:

In [ ]:
class Model(Compartment):
    A: Species = initial(default=1)
    B: Species = initial(default=0)
    r = MassAction(reactants=[2 * A], products=[B], rate=1)


Simulator(Model).solve(save_at=np.linspace(0, 10, 100)).plot()

### Predefined reactions

In the `simbio.reactions`,
we included several commoon reactions,
which simplify reading a model.

For instance,
for a Michaelis-Menten reaction:
`E + S <-> ES -> P + E`
we can write:

In [ ]:
from simbio import reactions


class Model(Compartment):
    A: Species = initial(default=1)
    B: Species = initial(default=0)
    r = reactions.MichaelisMenten(
        S=A,
        P=B,
        E=1,  # initial condition for internal variable
        ES=0,
        forward_rate=1,
        reverse_rate=0.2,
        catalytic_rate=0.1,
    )


Simulator(Model).solve(save_at=np.linspace(0, 100, 100)).plot()

### BioModels and SBML

In the `simbio.io.sbml` module,
we include an `SBML` importer.

We also include a BioModels importer,
which downloads (and caches) the model from the BioModels
and then imports the SBML file.

In [ ]:
from simbio.io import biomodels

model = biomodels.load_model("MODEL2105210001")

In [ ]:
model

In [ ]:
sim = Simulator(model)

sim.solve(save_at=np.linspace(0, 30e3, 1000)).plot(legend=False)

### Faster solve by switching backends

By default, `poincare` (hence `simbio`) use `NumPy` as the default backend.

But it supports different backends:

- NumPy
- Numba
- JAX

And it should be easy to add new ones.

Switching to the `numba` backend,
reduces the simulation speed considerably:

In [ ]:
sim = Simulator(model)

%timeit -n1 -r1 sim.solve(save_at=np.linspace(0, 30e3, 1000))
numpy_time = %timeit -n1 -r1 -o sim.solve(save_at=np.linspace(0, 30e3, 1000))
None

In [ ]:
sim = Simulator(model, backend="numba")

numba_compile_time = %timeit -n1 -r1 -o sim.solve(save_at=np.linspace(0, 30e3, 1000))
numba_run_time = %timeit -n1 -r1 -o sim.solve(save_at=np.linspace(0, 30e3, 1000))
None

In [ ]:
print(f"Speed up: {numpy_time.best / numba_run_time.best:.0f}x")
print(f"Need at least {numba_compile_time.best / numpy_time.best:.1f} runs")

### Changing solvers

The default solver is LSODA,
which works well in most situations,
but all `SciPy` solvers are available in the `solver` module:

In [ ]:
from poincare import solvers

sim.solve(
    save_at=np.linspace(0, 30e3, 1000),
    solver=solvers.RK45(),
).plot(legend=False)

As RK45 is not suitable for stiff equations,
it takes a much longer time to solve.